## Analyze RV chains

In [4]:
from analyzeRV import analyze_rv_chains

In [5]:
RVchain_files = ['chain_test_K03835482.csv', 'chain_test_K06233093.csv', 'chain_test_K08145411.csv', 'chain_test_K12254688.csv']
# the file with the MCMC chain results
for ii in range(0,4):
    RVchain = RVchain_files[ii]  
    print('')
    print('')
    print('')
    print(RVchain)
    print('--------')
    

    best,meds,x = analyze_rv_chains(RVchain)
    
    
    # output the median and 1-sigma error results to a TeX file
    # use None if not desired


    #labels
    labels = ['$P$ [days]', '$t_{tran}$ [BJD - 2454833]', '$\sqrt{e} cos\omega$', '$\sqrt{e} sin\omega$',
              '$K$ [km/s]', '$\gamma [km/s]$', '$\sigma_{j} [km/s]$']



    rv_time, rv_val, rv_unc = rvs[ii][1:4]

    texout = './plots/latex/' + RVchain[17:-4] + '.tex'

    foldername = './plots/'
    cornerFigname = 'corner_' + RVchain[17:-4] + '.png'
    chainFigname = 'chainPlot_'+ RVchain[17:-4] + '.png'
    RVfigname_meds = 'RVfit_meds_' + RVchain[17:-4] + '.png'
    RVfigname_best = 'RVfit_best_' + RVchain[17:-4] + '.png'

    numObs = len(rv_time)
    print('numObs', numObs)


    # iteration where burn-in stops
    burnin = 2000
    # make the triangle plot
    maketriangle = True



    print(plot_foldedRV(meds, rv_time, rv_val, rv_unc, foldername + RVfigname_meds))
    print(plot_foldedRV(best, rv_time, rv_val, rv_unc, foldername + RVfigname_best))

    rms_meds = get_RMS_residuals(meds, rv_time, rv_val, rv_unc)
    print('rms meds', rms_meds)

    rms_best = get_RMS_residuals(best, rv_time, rv_val, rv_unc)
    print('rms best', rms_best)

    redChisQ_meds = (loglikelihood(
        meds, rv_time, rv_val, rv_unc, chisQ=True) /
        (numObs - len(meds)))

    print('Reduced chi-square medians: ',  redChisQ_meds)


    redChisQ_best = (loglikelihood(
        best, rv_time, rv_val, rv_unc, chisQ=True) /
        (numObs - len(best)))

    print('Reduced chi-square best: ',  redChisQ_best)


    # put the MCMC results into a TeX table
    if texout is not None:
        best_out = best.copy()
        best_out = list(best_out)

        # calculate eccentricity and add it to the list of parameters
        e = (x[:, 2]**2. + x[:, 3]**2.).reshape((len(x[:, 0]), 1))
        e_best = best[2]**2. + best[3]**2.
        best_out.append(e_best)
        x = np.concatenate((x, e), axis=1)
        labels.append('$e$')

        # add omega to the list
        omega = np.arctan2(x[:, 3], x[:, 2]).reshape((len(x[:, 0]), 1))*180./np.pi
        omega_best = np.arctan2(best[3], best[2])*180./np.pi
        best_out.append(omega_best)
        x = np.concatenate((x, omega), axis=1)
        labels.append('$\omega$ [deg]')
        
        
        # what are the median and 1-sigma limits of each parameter we care about
        stds = [15.87, 50., 84.13]
        neg1, med, plus1 = np.percentile(x, stds, axis=0)



        # get ready to write them out
        ofile = open(texout, 'w')
        ofile.write('\\documentclass{article}\n\\usepackage{graphicx}\n\\usepackage[margin=1in]{geometry}\n\n\\begin{document}\n\n')
        ofile.write('\\begin{table}\n\\centering\n')
        ofile.write('\\caption{Median Reduced $\\chi^2$: ' + str(np.round(redChisQ_meds, decimals = 2)) + ' -- Maximum-Likelihood Reduced $\\chi^2$: ' + str(np.round(redChisQ_best, decimals = 2)) + '}\n')
        ofile.write('\\begin{tabular}{| c | c | c |}\n\\hline\n')

        # what decimal place the error bar is at in each direction
        sigfigslow = np.floor(np.log10(np.abs(plus1-med)))
        sigfigshigh = np.floor(np.log10(np.abs(med-neg1)))
        sigfigs = sigfigslow * 1
        # take the smallest of the two sides of the error bar
        lower = np.where(sigfigshigh < sigfigs)[0]
        sigfigs[lower] = sigfigshigh[lower]
        # go one digit farther
        sigfigs -= 1
        # switch from powers of ten to number of decimal places
        sigfigs *= -1.
        sigfigs = sigfigs.astype(int)

        # go through each parameter
        ofile.write('Parameter & Median and $1 \sigma$ Values & Maximum-Likelihood \\\\\n\\hline\n')
        for ii in np.arange(len(labels)):

            # if we're rounding to certain decimal places, do it
            if sigfigs[ii] >= 0:
                val = '%.'+str(sigfigs[ii])+'f'
            else:
                val = '%.0f'
            # do the rounding to proper decimal place and write the result
            ostr = labels[ii]+' & $'
            ostr += str(val % np.around(med[ii], decimals=sigfigs[ii]))
            ostr += '^{+' + str(val % np.around(plus1[ii]-med[ii],
                                                decimals=sigfigs[ii]))
            ostr += '}_{-' + str(val % np.around(med[ii]-neg1[ii],
                                                 decimals=sigfigs[ii]))

            best_val = round(best_out[ii], sigfigs[ii])
            ostr += '}$ & $' + str(best_val)

            ostr += '$ \\\\\n\\hline\n'
            ofile.write(ostr)

        ofile.write('\\end{tabular}\n\\end{table}\n\n')


        ofile.write('\\clearpage\n\n')
        ofile.write('\\begin{figure}[!htb]\n\\centering\n\\includegraphics[width=0.9\\textwidth]{../' + str(RVfigname_meds) + '}\n\\caption{Phase folded median MCMC RV model and observations. RMS residual velocity of ' + str(np.round(rms_meds, decimals = 2)) + ' $\\rm{km \\: s^{-1}}$.}\n\n')
        ofile.write('\\includegraphics[width=0.9\\textwidth]{../' + str(RVfigname_best) + '}\n\\caption{Phase folded maximum-likelihood MCMC RV model and observations. RMS residual velocity of ' + str(np.round(rms_best, decimals = 2)) + ' $\\rm{km \\: s^{-1}}$.}\n\\end{figure}\n\n\n')
        
        ofile.write('\\begin{figure}[!htb]\n\\centering\n\\includegraphics[width=0.9\\textwidth]{../unfoldedRV/' + str(RVfigname_meds) + '}\n\\caption{Time series median MCMC RV model and observations. RMS residual velocity of ' + str(np.round(rms_meds, decimals = 2)) + ' $\\rm{km \\: s^{-1}}$.}\n\n')
        ofile.write('\\includegraphics[width=0.9\\textwidth]{../unfoldedRV/' + str(RVfigname_best) + '}\n\\caption{Time series maximum-likelihood MCMC RV model and observations. RMS residual velocity of ' + str(np.round(rms_best, decimals = 2)) + ' $\\rm{km \\: s^{-1}}$.}\n\\end{figure}\n\n\n')
        
        
        ofile.write('\\begin{figure}[!htb]\n\\centering\n\\includegraphics[width=\\textwidth]{../' + str(cornerFigname) + '}\n\\caption{Contour plots showing the $1 \\sigma$, $2 \\sigma$, and $3 \\sigma$ constraints on pairs of parameters for the MCMC model.}\n\\end{figure}\n\n\n')
        ofile.write('\\begin{figure}[!htb]\n\\centering\n\\includegraphics[width=\\textwidth]{../' + str(chainFigname) + '}\n\\caption{MCMC chains for all 50 walkers. Blue line is burnout: ' + str(burnin) + ' steps.}\n\\end{figure}\n\n')

        ofile.write('\\end{document}')
        ofile.close()
        
        
    print(plot_RV(meds, rv_time, rv_val, rv_unc, foldername + 'unfoldedRV/' + RVfigname_meds))
    print(plot_RV(best, rv_time, rv_val, rv_unc, foldername + 'unfoldedRV/' + RVfigname_best))




chain_test_K03835482.csv
--------
File loaded


ValueError: zero-size array to reduction operation minimum which has no identity